In [ ]:
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import hopsworks
from helpers import util
import json

import warnings
warnings.filterwarnings("ignore")

In [ ]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
sensors_str = secrets.get_secret("SENSORS_JSON").value
sensors_data = json.loads(sensors_str)

In [ ]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [ ]:
# Select features for training data. todo check if city should be here
base_features = air_quality_fg.select(['pm25', 'city', 'date']).join(weather_fg.select_features(), on=['city'])
lagged_features = air_quality_fg.select(['pm25', 'lagged_1', 'lagged_2', 'lagged_3', 'city', 'date']).join(weather_fg.select_features(), on=['city'])

In [ ]:
base_feature_view = fs.get_or_create_feature_view(
    name='air_quality_base_fv',
    description="weather features with air quality as the target",
    version=1,
    labels=['pm25'],
    query=base_features,
)

lagged_feature_view = fs.get_or_create_feature_view(
    name='air_quality_lagged_fv',
    description="weather features with air quality as the target",
    version=1,
    labels=['pm25'],
    query=lagged_features,
)

In [ ]:
start_date_test_data = "2025-05-01"
# Convert string to datetime object
test_start = datetime.strptime(start_date_test_data, "%Y-%m-%d")

In [ ]:
X_base_train, X_base_test, y_base_train, y_base_test = base_feature_view.train_test_split(
    test_start=test_start
)

X_lagged_train, X_lagged_test, y_lagged_train, y_lagged_test = lagged_feature_view.train_test_split(
    test_start=test_start
)

In [ ]:
X_base_train['city'] = X_base_train["city"].astype("category")
X_base_test['city'] = X_base_test["city"].astype("category")
X_base_features = X_base_train.drop(columns=['date'])
X_base_test_features = X_base_test.drop(columns=['date'])

X_lagged_train['city'] = X_lagged_train["city"].astype("category")
X_lagged_test['city'] = X_lagged_test["city"].astype("category")
X_lagged_features = X_lagged_train.drop(columns=['date'])
X_lagged_test_features = X_lagged_test.drop(columns=['date'])

In [ ]:
# Creating an instance of the XGBoost Regressor
xgb_base_regressor = XGBRegressor(enable_categorical=True, tree_method="hist")
xgb_lagged_regressor = XGBRegressor(enable_categorical=True, tree_method="hist")

# Fitting the XGBoost Regressor to the training data
xgb_base_regressor.fit(X_base_features, y_base_train)
xgb_lagged_regressor.fit(X_lagged_features, y_lagged_train)


In [ ]:
# Predicting target values on the test set
y_base_pred = xgb_base_regressor.predict(X_base_test_features)
y_lagged_pred = xgb_lagged_regressor.predict(X_lagged_test_features)

# Calculating Mean Squared Error (MSE) using sklearn
base_mse = mean_squared_error(y_base_test.iloc[:,0], y_base_pred)
print("Base MSE:", base_mse)

# Calculating R squared using sklearn
base_r2 = r2_score(y_base_test.iloc[:,0], y_base_pred)
print("Base R squared:", base_r2)

# Calculating Mean Squared Error (MSE) using sklearn
lagged_mse = mean_squared_error(y_lagged_test.iloc[:,0], y_lagged_pred)
print("Lagged MSE:", lagged_mse)

# Calculating R squared using sklearn
lagged_r2 = r2_score(y_lagged_test.iloc[:,0], y_lagged_pred)
print("Lagged R squared:", lagged_r2)

In [ ]:
base_df = y_base_test
base_df['city'] = X_base_test['city']
base_df['date'] = X_base_test['date']
base_df['predicted_pm25'] = y_base_pred

lagged_df = y_lagged_test
lagged_df['city'] = X_lagged_test['city']
lagged_df['date'] = X_lagged_test['date']
lagged_df['predicted_pm25'] = y_lagged_pred

In [ ]:
# Creating a directory for the model artifacts if it doesn't exist
base_model_dir = "air_quality_model_base"
if not os.path.exists(base_model_dir):
    os.mkdir(base_model_dir)
base_images_dir = base_model_dir + "/images"
if not os.path.exists(base_images_dir):
    os.mkdir(base_images_dir)

lagged_model_dir = "air_quality_model_lagged"
if not os.path.exists(lagged_model_dir):
    os.mkdir(lagged_model_dir)
lagged_images_dir = lagged_model_dir + "/images"
if not os.path.exists(lagged_images_dir):
    os.mkdir(lagged_images_dir)

In [ ]:
for location in sensors_data:
    city = location['city']
    street = location['street']
    base_city_dir = f"{base_images_dir}/{city}"
    lagged_city_dir = f"{lagged_images_dir}/{city}"
    if not os.path.exists(base_city_dir):
        os.mkdir(base_city_dir)
    if not os.path.exists(lagged_city_dir):
        os.mkdir(lagged_city_dir)
    base_file_path = f"{base_city_dir}/pm25_hindcast.png"
    lagged_file_path = f"{lagged_city_dir}/pm25_hindcast.png"
    base_plt = util.plot_air_quality_forecast(city, street, base_df[base_df['city'] == city], base_file_path, hindcast=True) 
    lagged_plt = util.plot_air_quality_forecast(city, street, lagged_df[lagged_df['city'] == city], lagged_file_path, hindcast=True) 
    base_plt.show()
    lagged_plt.show()

In [ ]:
# Plotting feature importances using the plot_importance function from XGBoost
plot_importance(xgb_base_regressor)
base_feature_importance_path = base_images_dir + "/feature_importance.png"
plt.savefig(base_feature_importance_path)
plt.show()
plot_importance(xgb_lagged_regressor)
lagged_feature_importance_path = lagged_images_dir + "/feature_importance.png"
plt.savefig(lagged_feature_importance_path)
plt.show()

In [ ]:
# Saving the XGBoost regressor object as a json file in the model directory
xgb_base_regressor.save_model(base_model_dir + "/model.json")
xgb_lagged_regressor.save_model(lagged_model_dir + "/model.json")

In [ ]:
base_res_dict = { 
        "MSE": str(base_mse),
        "R squared": str(base_r2),
    }

lagged_res_dict = { 
        "MSE": str(lagged_mse),
        "R squared": str(lagged_r2),
    }

In [ ]:
mr = project.get_model_registry()

# Creating a Python model in the model registry named 'air_quality_xgboost_model'

base_aq_model = mr.python.create_model(
    name="air_quality_xgboost_base_model", 
    metrics= base_res_dict,
    feature_view=base_feature_view,
    description="Air Quality (PM2.5) predictor",
)

lagged_aq_model = mr.python.create_model(
    name="air_quality_xgboost_lagged_model", 
    metrics= lagged_res_dict,
    feature_view=lagged_feature_view,
    description="Air Quality (PM2.5) predictor",
)

# Saving the model artifacts to the 'air_quality_model' directory in the model registry
base_aq_model.save(base_model_dir)
lagged_aq_model.save(lagged_model_dir)